<a href="https://cloudevel.com"> <img src="img/cloudevel.png" width="500px"></a>

# Ejemplo de un proyecto de Gradle.

## Preliminares.

Antes de empezar, es necesario instalar y configurar algunas herramientas básicas. La siguiente celda verificará que *Gradle* esté instalado en el sistema.

In [ ]:
gradle -v

## Comandos de *Gradle*.

Gradle permite probar, compilar y construir un proyecto mediante una serie de comandos, los cuales peuden ser consultados en la siguiente liga.

https://docs.gradle.org/current/userguide/command_line_interface.html

## *Spring Boot.*

[*Spring Boot*](https://spring.io/projects/spring-boot) es una herramienta que forma parte de [*Spring Framework*](https://spring.io), la cual permite desarrollar y desplegar aplicaciones web de forma rápida y con muy pocas configuraciones.

* Es compatible con [*Apache Maven*](https://maven.apache.org/) y [*Gradle*](https://gradle.org/).
* Permite utilizar código escrito en *Java*, [*Groovy*](https://groovy-lang.org/) y [*Kotlin*](https://kotlinlang.org/).
* Cuenta con todo el stack de *Spring* para desarrollo de aplicaciones web.
* Cuenta con una interfaz de línea de comandos (CLI).
* Permite crear plantillas de proyectos de forma automatizada.


La documentación de referencia de Spring Boot puede ser consultada desde:
https://docs.spring.io/spring-boot/docs/2.1.12.RELEASE/reference/html/

* La siguiente celda despelgará un servicio web en http://localhost:8080 a partir del archivo previo.

### *Spring Initializr*.

Para facilitar el desarrollo de plantillas para el desarrollo de aplicaciones a la medida de forma ágil, está disponible el servico en línea llamado *Spring Initializr*, mediante el cual es posible crear una estructura de archivos compatibles con *Apache Maven* y *Gradle* que incluyen los componentes y dependencias de un proyecto específico.

El servicio se encuentra en https://start.spring.io/

Al final se podrá descargar un archivo comp-rimido que contiene lo necesario para comenzar a desarrolla una aplicación.

## Despliegue de un proyecto de *API REST* simple.

Se creará el archivo *demo-rest.zip* contiene una estructura de directorios y archivos creada mediante [*Spring Initializr*](https://start.spring.io/).

* Se utilizará *Gradle* para construir un archivo *.jar*.
* Se seleccionó *Groovy* como el lenguaje por defecto de la aplicación. 
* El grupo del proyecto es ```com.cloudevel```.
* El nombre del proyecto es ```demo-rest```.
* Se añadirá el componente de *Spring Web*.
* La aplicación utiliza una estructura [*HashMap*](https://docs.oracle.com/javase/8/docs/api/java/util/HashMap.html) para almacenar objetos que contienen los atributos:
    * ```id```
    * ```name```

* A continuación se creará el proyecto ```demo-rest```.

<img src="img/15/initializr.png">

### Definción del proyecto.

* La *API REST* corresponde a un sistema *CRUD* para objetos instanciados de una clase ```Product```.
* El endpoint [localhost:8080/products](localhost:8080/products) tiene habilitados los métodos:
   * ```GET``` para obtener un listado de los productos en formato *JSON*.
   * ```POST``` el cual dará de alta un nuevo producto al recibir datos en formato *JSON* con la estructura ```{"id":"<número>", "name":"<nombre>"}```.
* El endpoint ```localhost:800/products/<id>```, donde ```<id>``` es un número de identificación, tiene habilitados los métodos.
   * ```PUT``` el cual modificará a un producto al recibir datos en formato *JSON* con la estructura ```{"name":"<nombre>"}```.
   * ```DELETE``` el cual eliminará al producto con el valor ```<id>``` correspondiente.
* Los datos serán almacenados y gestionados en una estructura ```HashMap```.

* A continuación se desempaquetará el proyecto ```demo-rest```.

In [ ]:
unzip demo-rest.zip

* El paquete contiene una estructura similar a la siguiente:

```
demo-rest
├── HELP.md
├── build.gradle
├── gradle
│   └── wrapper
│       ├── gradle-wrapper.jar
│       └── gradle-wrapper.properties
├── gradlew
├── gradlew.bat
├── settings.gradle
└── src
    ├── main
    │   ├── groovy
    │   │   └── com
    │   │       └── cloudevel
    │   │           └── demorest
    │   │               └── DemoRestApplication.groovy
    │   └── resources
    │       ├── application.properties
    │       ├── static
    │       └── templates
    └── test
        └── groovy
            └── com
                └── cloudevel
                    └── demorest
                        └── DemoRestApplicationTests.groovy

16 directories, 10 files
```

In [ ]:
tree demo-rest

### El archivo ```build.gradle```.

``` groovy
plugins {
	id 'org.springframework.boot' version '2.2.6.RELEASE'
	id 'io.spring.dependency-management' version '1.0.9.RELEASE'
	id 'groovy'
}

group = 'com.cloudevel'
version = '0.0.1-SNAPSHOT'
sourceCompatibility = '11'

repositories {
	mavenCentral()
}

dependencies {
	implementation 'org.springframework.boot:spring-boot-starter-web'
	implementation 'org.codehaus.groovy:groovy'
	testImplementation('org.springframework.boot:spring-boot-starter-test') {
		exclude group: 'org.junit.vintage', module: 'junit-vintage-engine'
	}
}

test {
	useJUnitPlatform()
}

```

### El archivo ```DemoRestApplication.groovy```.

``` groovy
package com.cloudevel.demorest

import org.springframework.boot.SpringApplication
import org.springframework.boot.autoconfigure.SpringBootApplication

@SpringBootApplication
class DemoRestApplication {

	static void main(String[] args) {
		SpringApplication.run(DemoRestApplication, args)
	}

}
```

### El archivo ```ProductServiceController.java```.

Este archivo es el encargado de la parte del controlador de *MVC*, realziando operaciones específicas de un *endpoint* en función del métido utilizado para acceder a este. 


```java
package com.cloudevel.demorest.controller;

import java.util.HashMap;
import java.util.Map;

import org.springframework.http.HttpStatus;
import org.springframework.http.ResponseEntity;
import org.springframework.web.bind.annotation.PathVariable;
import org.springframework.web.bind.annotation.RequestBody;
import org.springframework.web.bind.annotation.RequestMapping;
import org.springframework.web.bind.annotation.RequestMethod;
import org.springframework.web.bind.annotation.RestController;

import com.cloudevel.demorest.model.Product;

@RestController
public class ProductServiceController {
   private static Map<String, Product> productRepo = new HashMap<>();
   static {
      Product leche = new Product();
      leche.setId("1");
      leche.setName("Leche");
      productRepo.put(leche.getId(), leche);
      
      Product pan = new Product();
      pan.setId("2");
      pan.setName("Pan de caja");
      productRepo.put(pan.getId(), pan);
   }
   
   @RequestMapping(value = "/products/{id}", method = RequestMethod.DELETE)
   public ResponseEntity<Object> delete(@PathVariable("id") String id) { 
      productRepo.remove(id);
      return new ResponseEntity<>("Producto eliminado.", HttpStatus.OK);
   }
   
   @RequestMapping(value = "/products/{id}", method = RequestMethod.PUT)
   public ResponseEntity<Object> updateProduct(@PathVariable("id") String id, @RequestBody Product product) { 
      productRepo.remove(id);
      product.setId(id);
      productRepo.put(id, product);
      return new ResponseEntity<>("Producto actualizado.", HttpStatus.OK);
   }
   
   @RequestMapping(value = "/products", method = RequestMethod.POST)
   public ResponseEntity<Object> createProduct(@RequestBody Product product) {
      productRepo.put(product.getId(), product);
      return new ResponseEntity<>("Producto creado.", HttpStatus.CREATED);
   }
   
   @RequestMapping(value = "/products")
   public ResponseEntity<Object> getProduct() {
      return new ResponseEntity<>(productRepo.values(), HttpStatus.OK);
   }
}
```

### El paquete ```org.springframework.http```.

Este paquete contiene herramientas par la gestión de peticiones y respuestas *HTTP*.

https://docs.spring.io/spring-framework/docs/current/javadoc-api/org/springframework/http/

#### La clase ```org.springframework.http.ResponseEntity```.

https://docs.spring.io/spring-framework/docs/current/javadoc-api/org/springframework/http/ResponseEntity.html

#### La clase ```org.springframework.http.HttpStatus```.

https://docs.spring.io/spring-framework/docs/current/javadoc-api/org/springframework/http/HttpStatus.html

## El archivo *Product.java*.

``` java
package com.cloudevel.demorest.model;

public class Product {
   private String id;
   private String name;

   public String getId() {
      return id;
   }
   public void setId(String id) {
      this.id = id;
   }
   public String getName() {
      return name;
   }
   public void setName(String name) {
      this.name = name;
   }
}
```

* Se copiarán los archivos a ```demo-rest/src/main/groovy/com/cloudevel/demorest/```.

In [ ]:
cp src/15/DemoRestApplication.groovy src/15/Product.java src/15/ProductServiceController.java demo-rest/src/main/groovy/com/cloudevel/demorest/

* Se creará el archivo *.jar*.

In [ ]:
cd demo-rest

* La siguiente celda ejecutará el comando de gradle para crear el archivo *.jar*, el cual estará localizado en ```demo-rest/build/libs/demo-rest-0.0.1-SNAPSHOT.jar```.

In [ ]:
gradle clean build

* Se ejecutará dicho archivo, el cual levantará un servicio que podrá ser consultado en http://localhost:8080/products

In [ ]:
java -jar build/libs/demo-rest-0.0.1-SNAPSHOT.jar

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2020.</p>